In [306]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math
import warnings

In [2]:
start = time.time()
# import nodes
road_node = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\nodes.shp')
road_node = road_node.set_index('osmid')
road_nodes = road_node.to_crs(4326)
road_nodes['geometry_m'] = gpd.GeoSeries(road_nodes['geometry'], crs = 4326).to_crs(3043)
road_nodes

,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m
osmid,,,,,,,,,
103237949,4.431781e+06,503033.785157,3,-74.964440,40.036252,None,None,POINT (-74.96444 40.03625),POINT (-5689489.546 8448457.709)
103353127,4.431724e+06,502937.820011,3,-74.965565,40.035733,None,None,POINT (-74.96557 40.03573),POINT (-5689617.107 8448567.833)
103353090,4.431839e+06,503124.706820,3,-74.963374,40.036770,None,None,POINT (-74.96337 40.03677),POINT (-5689364.441 8448354.775)
103237976,4.431638e+06,503122.315306,4,-74.963403,40.034959,None,None,POINT (-74.96340 40.03496),POINT (-5689653.040 8448263.511)
103238007,4.431510e+06,503203.675365,4,-74.962450,40.033810,None,None,POINT (-74.96245 40.03381),POINT (-5689797.056 8448087.089)
...,...,...,...,...,...,...,...,...,...
9703813717,4.416438e+06,483689.901846,3,-75.190791,39.897861,None,None,POINT (-75.19079 39.89786),POINT (-5720575.973 8468970.364)
9703813743,4.416385e+06,483760.410069,3,-75.189965,39.897385,None,None,POINT (-75.18996 39.89738),POINT (-5720619.056 8468844.195)
9703813748,4.416433e+06,484049.808886,3,-75.186581,39.897820,None,None,POINT (-75.18658 39.89782),POINT (-5720414.758 8468450.854)


In [3]:
# import edges
road_edge = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\edges.shp')
road_edge = road_edge.set_index(['u','v','key'])
road_edges = road_edge.to_crs(4326)
road_edges['geometry_m'] = gpd.GeoSeries(road_edges['geometry'], crs = 4326).to_crs(3043)
road_edges

#Get both ways
RE = road_edges.reset_index()
RE2 = RE[['osmid','from','to','name','length','geometry']]
RE2.columns = ['osmid','to','from','name','length','geometry']
RE2 = RE2[['osmid','from','to','name','length','geometry']]
road_connections = pd.concat([RE, RE2]).to_crs(4326)
road_connections['key'] = road_connections['from'].astype(str) + '-' + road_connections['to'].astype(str)

In [4]:
# Get the graph object
graph = ox.graph_from_place('Philadelphia, United States', network_type="all", buffer_dist = 2000)
graph = ox.speed.add_edge_speeds(graph)
graph = ox.speed.add_edge_travel_times(graph)

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [305]:
# Import greenspace and population grids
Penn_bound = gpd.read_file(r'C:\Users\bartb\Downloads\Pennsylvania\boundaries.shp').to_crs(4326)
Philly_greenspace = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\Public_greenspace.gpkg').to_crs(4326)
Philly_popgrid = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\CPoPGrid.gpkg').to_crs(4326)
Philly_bound = Penn_bound[Penn_bound['municipal1'] == 'PHILADELPHIA'].to_crs(4326)

In [307]:
# Make sure only complete overlays progress and take their centroids
popgrid = Philly_popgrid.overlay(Philly_bound)
popgrid['centroid'] = popgrid.to_crs(4326).centroid
popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
popgrid['area'] = popgrid.area / popgrid.area.max()
popgrid = popgrid[(popgrid['area'] >= 0.99) & popgrid['PoP2015_Number'] > 0]
popgrid['grid_lon'] = popgrid['centroid_m'].x
popgrid['grid_lat'] = popgrid['centroid_m'].y
popgrid = popgrid.reset_index()
popgrid

,index,grid_id,PoP2015_Number,class_of_m,county,fed_aid_ur,fed_id_num,fips_area,fips_count,fips_mun_c,...,gpid,mslink,municipal,municipal1,geometry,centroid,centroid_m,area,grid_lon,grid_lat
0,6,259,113,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),0.996208,-5.675978e+06,8.459813e+06
1,7,260,135,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),0.996208,-5.675889e+06,8.459539e+06
2,13,400,137,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),0.996234,-5.676341e+06,8.459996e+06
3,16,403,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01003 40.13349, -75.01003 40.131...",POINT (-75.01115 40.13262),POINT (-5676072.561 8459176.309),0.996234,-5.676073e+06,8.459176e+06
4,17,404,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.00778 40.13349, -75.00778 40.131...",POINT (-75.00890 40.13262),POINT (-5675982.949 8458902.986),0.996234,-5.675983e+06,8.458903e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,10223,20883,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",POINT (-75.25819 39.88221),POINT (-5725754.488 8476450.010),0.999924,-5.725754e+06,8.476450e+06
3887,10225,20885,39,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",POINT (-75.25369 39.88221),POINT (-5725576.204 8475897.317),0.999924,-5.725576e+06,8.475897e+06
3888,10226,20886,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",POINT (-75.25145 39.88221),POINT (-5725487.039 8475620.982),0.999924,-5.725487e+06,8.475621e+06
3889,10275,21024,3,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",POINT (-75.26043 39.88048),POINT (-5726119.978 8476637.252),0.999949,-5.726120e+06,8.476637e+06


In [308]:
warnings.filterwarnings("ignore")

green_buffer = gpd.GeoDataFrame(geometry = Philly_greenspace.to_crs(3043).buffer(25).to_crs(4326))
Philly_greenspace['geometry_w_buffer'] = green_buffer
Philly_greenspace['geometry_w_buffer'] = gpd.GeoSeries(Philly_greenspace['geometry_w_buffer'], crs = 4326)
Philly_greenspace['geom buffer diff'] = Philly_greenspace['geometry_w_buffer'].difference(Philly_greenspace['geometry'])

# This function group components in itself that overlap (with the buffer set of 25 metres)
# https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
W = libpysal.weights.fuzzy_contiguity(Philly_greenspace['geometry_w_buffer'])
Philly_greenspace['components'] = W.component_labels
Philly_parks = Philly_greenspace.dissolve('components')

# Exclude parks below 0.04 ha.
Philly_parks = Philly_parks[Philly_parks.to_crs(3043).area > 400]

# Get the Philly buffer
Bound_buffer = gpd.GeoDataFrame(
    geometry = np.repeat(Philly_bound.to_crs(3043).
                         buffer(1000).to_crs(4326),len(Philly_parks)), 
    crs = 4326).reset_index().iloc[:,1]
Bound_buffer.index = Philly_parks.index
Philly_parks['bound_buffer'] = Bound_buffer
Philly_parks

In [10]:
# Take a buffer for the parks to ensure equal measurements for edge population grids
parks_within_range = Philly_parks[Philly_parks['geometry'].intersection(Philly_parks['bound_buffer']).area > 0]
parks_within_range = parks_within_range.loc[:, ~Philly_parks.columns.isin(['bound_buffer'])]
parks_within_range = parks_within_range.reset_index()
parks_within_range['centroid'] = parks_within_range.centroid
parks_within_range['centroid_m'] = parks_within_range['centroid'].to_crs(3043)
parks_within_range['geometry_w_buffer'] = gpd.GeoSeries(parks_within_range['geometry_w_buffer'], crs = 4326)
parks_within_range['geometry_w_buffer_m'] = parks_within_range['geometry_w_buffer'].to_crs(3043)
parks_within_range = gpd.GeoDataFrame(parks_within_range, geometry = 'geom buffer diff', crs = 4326)
parks_within_range

C:\Users\bartb\AppData\Local\Temp\ipykernel_18500\3404927833.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_within_range = Philly_parks[Philly_parks['geometry'].intersection(Philly_parks['bound_buffer']).area > 0]
C:\Users\bartb\AppData\Local\Temp\ipykernel_18500\3404927833.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_within_range['centroid'] = parks_within_range.centroid


,components,geometry,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,...,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,centroid,centroid_m,geometry_w_buffer_m
0,5,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,...,None,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.25608 40.06117, -75.25606 40.061...",POINT (-75.25165 40.05653),POINT (-5697693.589 8484600.730),"POLYGON ((-5697129.449 8485382.576, -5697129.4..."
1,6,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24691 39.91683, -75.24696 39.916...","POLYGON ((-75.24696 39.91680, -75.24697 39.916...",POINT (-75.24714 39.91942),POINT (-5719375.750 8477008.358),"POLYGON ((-5719780.450 8476846.530, -5719785.6..."
2,7,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24828 39.92126, -75.24836 39.921...","POLYGON ((-75.24836 39.92121, -75.24837 39.921...",POINT (-75.25640 39.96241),POINT (-5712881.953 8480355.991),"POLYGON ((-5719127.059 8477242.926, -5719138.2..."
3,8,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,Fee,Fee,FED,NPS,NPS,FED,NPS,...,Edgar Allan Poe National Historic Site,None,None,216.240707,2.114309e+03,"POLYGON ((-75.15037 39.96185, -75.15037 39.961...","POLYGON ((-75.15037 39.96186, -75.15038 39.961...",POINT (-75.15001 39.96195),POINT (-5708732.995 8467282.252),"POLYGON ((-5708763.625 8467321.486, -5708762.4..."
4,9,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,Fee,Fee,FED,NPS,NPS,FED,NPS,...,Independence National Historical Park,None,None,6162.483495,1.285207e+05,"POLYGON ((-75.17280 40.03368, -75.17280 40.033...","POLYGON ((-75.17280 40.03369, -75.17280 40.033...",POINT (-75.17237 40.03387),POINT (-5698171.846 8473736.251),"POLYGON ((-5698218.752 8473779.070, -5698217.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,734,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25744 39.94930, -75.25744 39.949...","POLYGON ((-75.25744 39.94930, -75.25806 39.949...",POINT (-75.25763 39.94953),POINT (-5714985.845 8479845.686),"POLYGON ((-5715014.449 8479809.842, -5715014.8..."
470,735,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27542 39.96896, -75.27542 39.968...","POLYGON ((-75.27542 39.96896, -75.27541 39.968...",POINT (-75.27612 39.96913),POINT (-5712590.186 8483121.334),"POLYGON ((-5712589.472 8483026.846, -5712589.5..."
471,736,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...",242023,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,458.291779,1.188739e+04,"POLYGON ((-75.25664 39.94598, -75.25669 39.945...","POLYGON ((-75.25669 39.94597, -75.25671 39.945...",POINT (-75.25609 39.94665),POINT (-5715384.150 8479507.849),"POLYGON ((-5715513.534 8479540.584, -5715516.4..."
472,739,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...",242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,...,None,None,None,552.520121,1.592591e+04,"POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27789 39.96685, -75.27788 39.966...",POINT (-75.27714 39.96644),POINT (-5713059.443 8483108.295),"POLYGON ((-5713024.922 8483221.016, -5713022.5..."


In [11]:
start_time = time.time()
ParkRoad = pd.DataFrame()
mat = list()
# For all
for i in range(len(parks_within_range)):
    dist = road_nodes['geometry'].to_crs(3043).distance(parks_within_range['geometry'].to_crs(
        3043)[i])
    buf_nodes = road_nodes[(dist < 25) & (dist > 0)]
    mat.append(list(np.repeat(i, len(buf_nodes))))
    ParkRoad = pd.concat([ParkRoad, buf_nodes])
    if i % 10 == 0: print(round(i/len(parks_within_range)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')

# Park no list conversion
mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

# Format
ParkRoad['Park_No'] = mat_u
ParkRoad = ParkRoad.reset_index()
ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
print('100 % done',round((time.time() - start_time) / 60,2),' mns')
ParkRoad

    

0.0 % done 0.01  mns
2.1 % done 0.25  mns
4.2 % done 0.34  mns
6.3 % done 0.59  mns
8.4 % done 0.89  mns
10.5 % done 1.06  mns
12.7 % done 1.27  mns
14.8 % done 1.6  mns
16.9 % done 1.67  mns
19.0 % done 1.74  mns
21.1 % done 1.81  mns
23.2 % done 1.87  mns
25.3 % done 1.94  mns
27.4 % done 2.01  mns
29.5 % done 2.09  mns
31.6 % done 2.16  mns
33.8 % done 2.22  mns
35.9 % done 2.3  mns
38.0 % done 2.38  mns
40.1 % done 2.45  mns
42.2 % done 2.54  mns
44.3 % done 2.6  mns
46.4 % done 2.67  mns
48.5 % done 2.74  mns
50.6 % done 2.81  mns
52.7 % done 2.87  mns
54.9 % done 2.92  mns
57.0 % done 2.98  mns
59.1 % done 3.04  mns
61.2 % done 3.1  mns
63.3 % done 3.18  mns
65.4 % done 3.26  mns
67.5 % done 3.32  mns
69.6 % done 3.39  mns
71.7 % done 3.47  mns
73.8 % done 3.55  mns
75.9 % done 3.62  mns
78.1 % done 3.69  mns
80.2 % done 3.76  mns
82.3 % done 3.84  mns
84.4 % done 3.91  mns
86.5 % done 3.99  mns
88.6 % done 4.07  mns
90.7 % done 4.15  mns
92.8 % done 4.23  mns
94.9 % done 4.31  m

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m,Park_No,park_lon,park_lat
0,109991185,4.433758e+06,479471.157101,3,-75.240687,40.053816,None,None,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,8.483120e+06
1,109991191,4.433867e+06,479608.431472,3,-75.239081,40.054803,None,None,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,8.482975e+06
2,5550942021,4.433928e+06,479315.926381,3,-75.242512,40.055347,None,None,POINT (-75.24251 40.05535),POINT (-5697523.197 8483422.258),0,-5.697523e+06,8.483422e+06
3,109991199,4.433975e+06,479743.480201,3,-75.237501,40.055776,None,None,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,8.482831e+06
4,109991204,4.434025e+06,479804.853265,3,-75.236783,40.056232,None,None,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,8.482767e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,110166321,4.424071e+06,476384.161037,1,-75.276527,39.966462,None,None,POINT (-75.27653 39.96646),POINT (-5713031.271 8483034.574),472,-5.713031e+06,8.483035e+06
7342,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),473,-5.713613e+06,8.482370e+06
7343,110133414,4.423531e+06,476586.279558,3,-75.274141,39.961600,None,None,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),473,-5.713713e+06,8.482492e+06
7344,923789083,4.423655e+06,476581.963819,3,-75.274196,39.962719,None,None,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),473,-5.713536e+06,8.482556e+06


In [180]:
# Save as file
ParkRoad[['Park_No','geometry']].to_crs(4326).to_file(r'C:\Users\bartb\Downloads\park_centroid.shp')

In [13]:
ParkRoad[ParkRoad['Park_No'] == 453]

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m,Park_No,park_lon,park_lat
7224,109890052,4.429296e+06,493169.700669,3,-75.080034,40.013838,None,None,POINT (-75.08003 40.01384),POINT (-5697684.279 8461406.828),453,-5.697684e+06,8.461407e+06
7225,110188472,4.429317e+06,493092.570321,3,-75.080938,40.014029,None,None,POINT (-75.08094 40.01403),POINT (-5697690.013 8461527.279),453,-5.697690e+06,8.461527e+06
7226,110291816,4.429273e+06,493150.989575,4,-75.080253,40.013627,None,None,POINT (-75.08025 40.01363),POINT (-5697726.578 8461422.644),453,-5.697727e+06,8.461423e+06


In [436]:
PtR = pd.merge(ParkRoad, parks_within_range[['geometry','geometry_w_buffer']], left_on = 'Park_No', right_index = True)
PtR['park_size_walkable'] = PtR['geometry_m'].buffer(500).to_crs(4326).intersection(PtR['geometry_y'])
PtR['walk_area'] = PtR['park_size_walkable'].to_crs(3043).area
PtR['park_area'] = PtR['geometry_y'].to_crs(3043).area
PtR['share_walked'] = PtR['walk_area'] / PtR['park_area']
PtR['size_infl_factor'] = PtR['walk_area'] / PtR['walk_area'].mean()
PtR['size_infl_proot2'] = PtR['size_infl_factor']**(1/2)
PtR['size_infl_proot4'] = PtR['size_infl_factor']**(1/4)
PtR

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry_x,geometry_m,...,park_lat,geometry_y,geometry_w_buffer,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot4
0,109991185,4.433758e+06,479471.157101,3,-75.240687,40.053816,None,None,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),...,8.483120e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.23914 40.05656, -75.23880 40.056...",198723.068185,4.192678e+06,0.047398,1.504099,1.226417,1.107437
1,109991191,4.433867e+06,479608.431472,3,-75.239081,40.054803,None,None,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),...,8.482975e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24278 40.05572, -75.24265 40.055...",291442.444263,4.192678e+06,0.069512,2.205875,1.485219,1.218696
2,5550942021,4.433928e+06,479315.926381,3,-75.242512,40.055347,None,None,POINT (-75.24251 40.05535),POINT (-5697523.197 8483422.258),...,8.483422e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","MULTIPOLYGON (((-75.24062 40.05796, -75.24030 ...",440724.683283,4.192678e+06,0.105118,3.335766,1.826408,1.351447
3,109991199,4.433975e+06,479743.480201,3,-75.237501,40.055776,None,None,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),...,8.482831e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24077 40.05416, -75.24096 40.054...",245064.061025,4.192678e+06,0.058450,1.854846,1.361927,1.167016
4,109991204,4.434025e+06,479804.853265,3,-75.236783,40.056232,None,None,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),...,8.482767e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24005 40.05461, -75.24024 40.054...",187511.352028,4.192678e+06,0.044724,1.419240,1.191318,1.091475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,110166321,4.424071e+06,476384.161037,1,-75.276527,39.966462,None,None,POINT (-75.27653 39.96646),POINT (-5713031.271 8483034.574),...,8.483035e+06,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...","POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27647 39.96684, -75.27650 39.966...",19734.719624,1.973472e+04,1.000000,0.149369,0.386482,0.621677
7342,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),...,8.482370e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069,0.597553,0.773015
7343,110133414,4.423531e+06,476586.279558,3,-75.274141,39.961600,None,None,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),...,8.482492e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069,0.597553,0.773015
7344,923789083,4.423655e+06,476581.963819,3,-75.274196,39.962719,None,None,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),...,8.482556e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069,0.597553,0.773015


In [123]:
gpd.GeoDataFrame(PtR[['Park_No','park_size_walkable']], geometry = 'park_size_walkable', crs = 4326
                ).to_file(r'C:\Users\bartb\Downloads\parks.shp')

In [624]:
# Check all parks within 1000m radius
start_time = time.time()
len1 = len(popgrid)
len2 = len(ParkRoad)
len3 = int(np.ceil(len2/1000))
output = pd.DataFrame()
len_mat = 0
# Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
for i in range(len3):
    # Check all grid-park combinations per 1000
    l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
    listed = pd.DataFrame(list(product(l1, l2)))
    
    # Merge grid and park information
    grid_merged = pd.merge(listed, 
                           popgrid[['grid_id','centroid','centroid_m','grid_lon','grid_lat']],
                           left_on = 0, right_index = True)
    node_merged = pd.merge(grid_merged, 
                           PtR[['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                               'size_infl_factor','size_infl_proot2','size_infl_proot4','share_walked','park_area','walk_area']], 
                           left_on = 1, right_index = True)
    
    # Preset index for merging
    len4 = len(node_merged)
    node_merged['key'] = range(0,len(node_merged))
    node_merged = node_merged.set_index('key')
    node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]
        
    # Create lists for better computational performance
    glon = list(node_merged['grid_lon'])
    glat = list(node_merged['grid_lat'])
    plon = list(node_merged['park_lon'])
    plat = list(node_merged['park_lat'])
    infl = list(node_merged['size_infl_factor'])
    infl2 = list(node_merged['size_infl_proot2'])
    infl4 = list(node_merged['size_infl_proot4'])

    # Get the euclidean distances
    mat = list()
    mat2 = list()
    mat3 = list()
    mat4 = list()
    for j in range(len(node_merged)):
        mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
        mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl[j])
        mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
        mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl4[j])
        
    # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
    mat_df = pd.DataFrame(mat3)[(np.array(mat) <= 1000) | 
                                (np.array(mat2) <= 1000) | 
                                (np.array(mat3) <= 1000) | 
                                (np.array(mat4) <= 1000)]
    
    # join the other gravity euclidean scores and other information
    mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
    mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
    mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
    mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl','size_infl_eucl4']    
    mat_df = mat_df.join(node_merged)
        
    output = pd.concat([output, mat_df])

    print((i+1),'/',len3,'done',round((time.time() - start_time) / 60,2),' mns')
    print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within a 1000m threshold:',len(mat_df))
    
    # Checks the number of the parks within 1000m.
    len_mat = len_mat + len(mat_df)
    
# Renaming columns
print('total combinations within distance',len_mat)
output.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl','size_infl_eucl4',
                  'Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'grid_lon','grid_lat',
                  'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                  'park_lon','park_lat','size_infl_factor','size_infl_proot2','size_infl_proot4',
                  'parkshare_walked','park_area','walk_area_m2']

output = output[['raw euclidean','size_infl_eucl','size_infl_eucl2','size_infl_eucl4',
                 'Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                 'walk_area_m2','size_infl_factor','size_infl_proot2','size_infl_proot4']]
output

1 / 8 done 0.4  mns
of 3891000 within a 1000m threshold: 53156
2 / 8 done 0.81  mns
of 3891000 within a 1000m threshold: 33645
3 / 8 done 1.22  mns
of 3891000 within a 1000m threshold: 58591
4 / 8 done 1.63  mns
of 3891000 within a 1000m threshold: 48283
5 / 8 done 2.03  mns
of 3891000 within a 1000m threshold: 18836
6 / 8 done 2.43  mns
of 3891000 within a 1000m threshold: 19864
7 / 8 done 2.84  mns
of 3891000 within a 1000m threshold: 18399
8 / 8 done 3.04  mns
of 346000 within a 1000m threshold: 4754
total combinations within distance 255528


,raw euclidean,size_infl_eucl,size_infl_eucl2,size_infl_eucl4,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_factor,size_infl_proot2,size_infl_proot4
868,1373.392587,913.099756,1119.841255,1240.153893,868,0,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.504099,1.226417,1.107437
913,1089.848567,724.585577,888.644222,984.117692,913,0,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.504099,1.226417,1.107437
914,1067.304885,709.597415,870.262482,963.761069,914,0,6265,POINT (-75.24022 40.06019),POINT (-5696662.252 8483389.707),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.504099,1.226417,1.107437
915,1121.233555,745.451880,914.235014,1012.457888,915,0,6266,POINT (-75.23797 40.06019),POINT (-5696573.814 8483114.956),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.504099,1.226417,1.107437
916,1241.708380,825.549540,1012.468164,1121.244935,916,0,6267,POINT (-75.23573 40.06019),POINT (-5696485.362 8482840.213),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.504099,1.226417,1.107437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181692,934.702623,1491.672341,1180.792128,1050.566276,2719,7303,12224,POINT (-75.25145 39.98767),POINT (-5708658.692 8481045.677),464,3618217765,POINT (-75.24980 39.99311),"POLYGON ((-75.24989 39.99303, -75.25100 39.992...",POINT (-5707727.165 8481122.665),82788.847320,0.626614,0.791589,0.889713
1181693,913.539094,1457.897908,1154.056642,1026.779362,2720,7303,12225,POINT (-75.24920 39.98767),POINT (-5708569.997 8480770.261),464,3618217765,POINT (-75.24980 39.99311),"POLYGON ((-75.24989 39.99303, -75.25100 39.992...",POINT (-5707727.165 8481122.665),82788.847320,0.626614,0.791589,0.889713
1248193,692.747670,28495.289917,4442.977122,1754.383667,3073,7320,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),468,3425873698,POINT (-75.26717 39.96453),"POLYGON ((-75.26652 39.96452, -75.26687 39.964...",POINT (-5712969.814 8481787.042),3211.987197,0.024311,0.155920,0.394867
1252084,716.791621,29484.307092,4597.184385,1815.274978,3073,7321,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),468,5544386939,POINT (-75.26643 39.96443),"POLYGON ((-75.26652 39.96452, -75.26687 39.964...",POINT (-5712957.523 8481690.948),3211.987197,0.024311,0.155920,0.394867


In [625]:
output[(output['Park_No'] == 231) & (output['Grid_No'] == 2639)]

,raw euclidean,size_infl_eucl,size_infl_eucl2,size_infl_eucl4,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_factor,size_infl_proot2,size_infl_proot4
434540,214.025883,481.159902,320.906019,262.072879,2639,5111,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809486,POINT (-75.11806 39.99394),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702367.869 8465026.271),58769.051358,0.444812,0.666943,0.816666
438431,208.087815,467.810302,312.002602,254.801766,2639,5112,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,5547581605,POINT (-75.11819 39.99395),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702370.518 8465042.292),58769.051358,0.444812,0.666943,0.816666
442322,315.122553,708.439254,472.488292,385.864894,2639,5113,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809494,POINT (-75.11790 39.99456),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702262.806 8465038.940),58769.051358,0.444812,0.666943,0.816666
446213,456.479714,1026.229780,684.436320,558.955540,2639,5114,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809504,POINT (-75.11773 39.99542),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702119.247 8465062.415),58769.051358,0.444812,0.666943,0.816666
450104,334.672172,752.389512,501.800590,409.803237,2639,5115,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556737,POINT (-75.11788 39.99468),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702242.736 8465042.228),58769.051358,0.444812,0.666943,0.816666
453995,480.640883,1080.547443,720.663082,588.540687,2639,5116,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109899816,POINT (-75.11966 39.99567),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702155.608 8465312.404),58769.051358,0.444812,0.666943,0.816666
457886,257.779648,579.524443,386.509517,315.648994,2639,5117,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109999351,POINT (-75.11999 39.99417),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702407.718 8465274.162),58769.051358,0.444812,0.666943,0.816666
461777,202.093837,454.335006,303.015354,247.462190,2639,5118,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556736,POINT (-75.11909 39.99406),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702389.554 8465157.699),58769.051358,0.444812,0.666943,0.816666


In [ ]:
park_entry_centroid = gpd.GeoDataFrame(output[['size_infl_eucl2','Grid_No','Park_No','Parkroad_osmid',
                                               'Parkroad_coords_centroid']],geometry = 'Parkroad_coords_centroid', crs = 4326)
park_entry_centroid.to_file(r'C:\Users\bartb\Downloads\park_centroid.shp')

In [626]:
start_time = time.time()

# Reinstate geographic elements
gp_entry = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
gp_entry['Grid_m_centroid'] = gpd.GeoSeries(gp_entry['Grid_m_centroid'], crs = 3043)
gp_entry['Parkroad_coords_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_coords_centroid'], crs = 4326)
gp_entry['Parkroad_m_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_m_centroid'], crs = 3043)

# Get index for merging later
roads = road_nodes.reset_index()

# Get the nearest entrance point for the grid centroids
mat5 = np.repeat(None,len(gp_entry))
for i in range(len(gp_entry)):
    nearest = int(roads['geometry'].sindex.nearest(gp_entry['Grid_coords_centroid'].iloc[i])[1])
    mat5[i] = roads['osmid'].iloc[nearest]
    if i % 100000 == 0: print(round(i/len(gp_entry)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')
print(len(mat5[mat5 != None]))

# Add all the information and the distance from the nearest point to the grid centroid in a geodataframe
pd.DataFrame(mat5)
gp_entry['grid_osm'] = mat5
gp_entry = pd.merge(gp_entry, road_nodes['geometry'], left_on = 'grid_osm', right_index = True)
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry['geometry_m'] = gp_entry['geometry'].to_crs(3043)
gp_entry['grid_entry_dist'] = round(gp_entry['Grid_m_centroid'].distance(gp_entry['geometry_m'].to_crs(3043)),3)
gp_entry = gp_entry.reset_index()
print('100 % done', round((time.time() - start_time) / 60,2),' mns')
gp_entry

0.0 % done 0.01  mns
39.1 % done 0.16  mns
78.3 % done 0.32  mns
255528
100 % done 0.43  mns


,index,raw euclidean,size_infl_eucl,size_infl_eucl2,size_infl_eucl4,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,...,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_factor,size_infl_proot2,size_infl_proot4,grid_osm,geometry,geometry_m,grid_entry_dist
0,868,1373.392587,913.099756,1119.841255,1240.153893,868,0,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.504099,1.226417,1.107437,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.08
1,4759,1266.184486,574.005428,852.523764,1038.966970,868,1,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697474.612 8482974.522),291442.444263,2.205875,1.485219,1.218696,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.08
2,8650,1098.225747,329.227465,601.303650,812.629774,868,2,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697523.197 8483422.258),440724.683283,3.335766,1.826408,1.351447,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.08
3,12541,1208.512478,651.543362,887.354655,1035.557421,868,3,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697257.444 8482831.074),245064.061025,1.854846,1.361927,1.167016,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.08
4,16432,1198.333947,844.349224,1005.888830,1097.902879,868,4,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697156.554 8482766.601),187511.352028,1.419240,1.191318,1.091475,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255523,855671,75.110843,250.261227,137.103361,101.478811,3542,7219,16514,POINT (-75.18407 39.93583),POINT (-5714253.505 8470109.063),...,"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714178.709 8470102.194),39653.440881,0.300130,0.547841,0.740163,967421639,POINT (-75.18392 39.93585),POINT (-5714244.099 8470091.719),19.73
255524,859562,65.590893,218.541780,119.726148,88.616844,3542,7220,16514,POINT (-75.18407 39.93583),POINT (-5714253.505 8470109.063),...,"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714187.982 8470112.045),39653.440881,0.300130,0.547841,0.740163,967421639,POINT (-75.18392 39.93585),POINT (-5714244.099 8470091.719),19.73
255525,863453,133.985107,446.423924,244.569330,181.021125,3542,7221,16514,POINT (-75.18407 39.93583),POINT (-5714253.505 8470109.063),...,"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714384.969 8470083.193),39653.440881,0.300130,0.547841,0.740163,967421639,POINT (-75.18392 39.93585),POINT (-5714244.099 8470091.719),19.73
255526,867344,123.221646,410.561233,224.922278,166.479108,3542,7222,16514,POINT (-75.18407 39.93583),POINT (-5714253.505 8470109.063),...,"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714374.248 8470084.472),39653.440881,0.300130,0.547841,0.740163,967421639,POINT (-75.18392 39.93585),POINT (-5714244.099 8470091.719),19.73


In [128]:
# Get only necessary road connections columns for network performance
road_conn = road_connections[['osmid','key','length','geometry']]
road_conn = road_conn.set_index('key')

In [627]:
warnings.filterwarnings("ignore")

start_time = time.time()

parknode = list(gp_entry['Parkroad_osmid'])
gridnode = list(gp_entry['grid_osm'])

s_mat = list([])
s_mat1 = list([])
s_mat2 = list([])
s_mat3 = list([])
s_mat4 = list([])

for i in range(len(gp_entry)):
    # First try from population grid to park Dijkstra algorithm
    try:
        shortest = nx.shortest_path(graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
        s_mat.append(shortest)
        shortest_to = list(shortest[1:len(shortest)])
        shortest_to.append(0)
        s_mat1.append(shortest_to)
        s_mat2.append(list(np.repeat(i, len(shortest))))
        s_mat3.append(list(np.arange(0, len(shortest))))
        s_mat4.append('normal way')
        
    except:
        try:
            # Check the reverse
            shortest = nx.shortest_path(graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
            s_mat.append(shortest)
            shortest_to = list(shortest[1:len(shortest)])
            shortest_to.append(0)
            s_mat1.append(shortest_to)
            s_mat2.append(list(np.repeat(i, len(shortest))))
            s_mat3.append(list(np.arange(0, len(shortest))))
            s_mat4.append('reverse way')
        except:
            try:
                # Get the route to the nearest park node
                new_node = pd.DataFrame((abs(gp_entry['Parkroad_coords_centroid'].x[i] - PtR['geometry_y'].x)**2
                                         + abs(gp_entry['Parkroad_coords_centroid'].y[i] - PtR['geometry_y'].y)**2
                                        )**(1/2)).join(PtR['osmid']).sort_values(0).iloc[1,1]
                shortest = nx.shortest_path(graph, gridnode[i], new_node, 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('nearest way')
            except:
                # Print if none of the above three options gives any result
                print('index',i,'no route possible between osmid',gridnode[i],'and',parknode[i])
                pass
            
    if i % 10000 == 0: print(round(i/len(gp_entry)*100,2),'% done', round((time.time() - start_time) / 60,2),' mns')
print('100 % pathfinding done', round((time.time() - start_time) / 60,2),' mns')


0.0 % done 0.01  mns
3.91 % done 0.26  mns
7.83 % done 0.74  mns
11.74 % done 1.57  mns
15.65 % done 2.03  mns
19.57 % done 2.63  mns
23.48 % done 2.97  mns
27.39 % done 3.26  mns
31.31 % done 3.72  mns
35.22 % done 4.12  mns
39.13 % done 4.69  mns
43.05 % done 5.12  mns
46.96 % done 5.41  mns
50.88 % done 6.2  mns
54.79 % done 6.58  mns
58.7 % done 6.95  mns
62.62 % done 7.65  mns
66.53 % done 8.0  mns
70.44 % done 8.39  mns
74.36 % done 8.93  mns
78.27 % done 9.51  mns
82.18 % done 10.48  mns
86.1 % done 11.41  mns
90.01 % done 11.72  mns
93.92 % done 11.91  mns
97.84 % done 12.02  mns
100 % pathfinding done 12.08  mns


In [628]:
start_time = time.time()
# Unpack lists and add them in a dataframe
s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()

# Format df
routes.columns = ['from','to','route','step']
mat_key = list([])
for i in range(len(routes)):
    mat_key.append(str(int(s_mat_u[i])) + '-' + str(int(s_mat_u1[i])))
routes['key'] = mat_key
routes = routes.set_index('key')

# Add route information
routes = routes.join(road_conn, how = 'left')
routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
routes.sort_values(by = ['route','step'])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes

time taken 2.25 mns


,from,to,route,step,osmid,length,geometry
key,,,,,,,
1015355888-0,1015355888,0,30942,23,NaN,NaN,None
1015355888-0,1015355888,0,31229,22,NaN,NaN,None
1015355888-0,1015355888,0,31480,27,NaN,NaN,None
1015355888-0,1015355888,0,34539,20,NaN,NaN,None
1015355888-0,1015355888,0,34843,17,NaN,NaN,None
...,...,...,...,...,...,...,...
989146009-2699409740,989146009,2699409740,201974,5,476000203,56.075,"LINESTRING (-75.18716 39.95470, -75.18705 39.9..."
989146009-2699409740,989146009,2699409740,202550,7,476000203,56.075,"LINESTRING (-75.18716 39.95470, -75.18705 39.9..."
989146009-7836188405,989146009,7836188405,192687,15,344740382,9.268,"LINESTRING (-75.18716 39.95521, -75.18705 39.9..."


In [629]:
start_time = time.time()
routegrid = list([])
for i in range(len(gp_entry)):
    routegrid.append(LineString([(gp_entry['geometry'][i]),(gp_entry['Grid_coords_centroid'][i])]))
gridline = gpd.GeoDataFrame(geometry = routegrid, crs = 4326)
gridline = gridline.join(gp_entry[['grid_entry_dist']])
gridline = gridline.reset_index()
gridline.columns = ['route','geometry','length']
gridline['grid_osm'] = '0-' + gp_entry['grid_osm'].astype(str)
gridline = gridline.set_index('grid_osm')
gridline['step'] = -1
gridline['osmid'] = np.nan
gridline['to'] = np.where(s_mat4 == 'normal way',gp_entry['Parkroad_osmid'],gp_entry['grid_osm'])

gridline['from'] = 0
gridline = gridline[['from','to','route','step','osmid','length','geometry']]
routes_stack = pd.concat([routes, gridline])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes_stack[routes_stack['route'] == 1].sort_values(by = 'step')

time taken 0.56 mns


,from,to,route,step,osmid,length,geometry
0-110562147,0,110562147,1,-1,NaN,44.080,"LINESTRING (-75.24257 40.06216, -75.24247 40.0..."
110562147-110546460,110562147,110546460,1,0,12205057,70.766,"LINESTRING (-75.24257 40.06216, -75.24294 40.0..."
110546460-110422537,110546460,110422537,1,1,"[12205057, 12182622]",206.378,"LINESTRING (-75.24499 40.06143, -75.24486 40.0..."
110422537-110422523,110422537,110422523,1,2,12182622,153.720,"LINESTRING (-75.24499 40.06143, -75.24566 40.0..."
110422523-110430427,110422523,110430427,1,3,12184719,50.287,"LINESTRING (-75.24687 40.06192, -75.24643 40.0..."
110430427-2281528216,110430427,2281528216,1,4,12187778,198.727,"LINESTRING (-75.24529 40.06060, -75.24645 40.0..."
2281528216-4774911885,2281528216,4774911885,1,5,12187778,137.150,"LINESTRING (-75.24418 40.05970, -75.24529 40.0..."
4774911885-4244000117,4774911885,4244000117,1,6,12187778,102.832,"LINESTRING (-75.24418 40.05970, -75.24343 40.0..."
4244000117-110443757,4244000117,110443757,1,7,12187778,232.152,"LINESTRING (-75.24336 40.05903, -75.24160 40.0..."
110443757-110305724,110443757,110305724,1,8,108593573,226.652,"LINESTRING (-75.24148 40.05751, -75.24177 40.0..."


In [792]:
start_time = time.time()
# get single (dissolved) line per route, attach information.
routes2 = routes_stack[['route','geometry']].dissolve('route')
routes2['way_calculated'] = s_mat4
routes2['route_cost'] = routes.groupby('route')['length'].sum()
routes2['num_steps'] = routes.groupby('route')['step'].max()
routes2.index = routes2.index.astype(int)
routes2 = pd.merge(routes2, gp_entry[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid','grid_osm',
                                      'walk_area_m2','size_infl_factor','size_infl_proot2', 'size_infl_proot4',
                                      'raw euclidean']],
                   left_index = True, right_index = True)
routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
routes2['grav_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_factor']
routes2['grav2_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot2']
routes2['grav4_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot4']

routes2['parkcost_ha'] = routes2['grav_total_cost'] * (routes2['walk_area_m2'] /10000)
routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes2 

time taken 1.32 mns


,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,...,size_infl_factor,size_infl_proot2,size_infl_proot4,raw euclidean,raw_total_cost,grav_total_cost,grav2_total_cost,grav4_total_cost,parkcost_km2,gridpark_no
route,,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,1726.207,11,868,0,0,44.08,109991185,110562147,...,1.504099,1.226417,1.107437,1373.392587,1770.287,1176.974918,1443.462295,1598.543880,233.892067,868-0
1,"MULTILINESTRING ((-75.23908 40.05480, -75.2406...",normal way,1901.505,12,868,0,1,44.08,109991191,110562147,...,2.205875,1.485219,1.218696,1266.184486,1945.585,882.001290,1309.965068,1596.448680,257.052612,868-0
2,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,1495.763,10,868,0,2,44.08,5550942021,110562147,...,3.335766,1.826408,1.351447,1098.225747,1539.843,461.616029,843.099170,1139.403508,203.445578,868-0
3,"MULTILINESTRING ((-75.23908 40.05480, -75.2406...",normal way,2074.101,13,868,0,3,44.08,109991199,110562147,...,1.854846,1.361927,1.167016,1208.512478,2118.181,1141.971469,1555.282054,1815.039641,279.856166,868-0
4,"MULTILINESTRING ((-75.23938 40.06072, -75.2397...",normal way,1456.034,13,868,0,4,44.08,109991204,110562147,...,1.419240,1.191318,1.091475,1198.333947,1500.114,1056.984237,1259.204849,1374.391073,198.196543,868-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255523,"MULTILINESTRING ((-75.18385 39.93619, -75.1839...",normal way,54.774,3,3542,452,7219,19.73,9454221984,967421639,...,0.300130,0.547841,0.740163,75.110843,74.504,248.239291,135.995662,100.658933,9.843542,3542-452
255524,"MULTILINESTRING ((-75.18385 39.93619, -75.1839...",normal way,57.045,3,3542,452,7220,19.73,9454221985,967421639,...,0.300130,0.547841,0.740163,65.590893,76.775,255.806018,140.141026,103.727177,10.143589,3542-452
255525,"MULTILINESTRING ((-75.18411 39.93502, -75.1842...",normal way,204.453,4,3542,452,7221,19.73,110306144,967421639,...,0.300130,0.547841,0.740163,133.985107,224.183,746.953574,409.211795,302.883357,29.619279,3542-452


In [815]:
# No gravity
start_time = time.time()

gridpark_raw = routes2.iloc[routes2.groupby('gridpark_no')['raw_total_cost'].idxmin()]
gridpark_raw = gridpark_raw.loc[:, ~gridpark_raw.columns.isin(['grav_total_cost','grav2_total_cost','grav4_total_cost',
                                                'size_infl_factor','size_infl_proot2','size_infl_proot4'])]
gridpark_raw = gridpark_raw[gridpark_raw['raw_total_cost'] <=1000]
gridpark_raw['score'] = 1000 - gridpark_raw['raw_total_cost']
gridpark_raw = pd.merge(gridpark_raw, popgrid[['PoP2015_Number','geometry']],
                        left_on = 'Grid_No', right_index = True, how = 'outer')
gridpark_raw['score'] = gridpark_raw['score'].fillna(0).round()
gridpark_raw['pop_score'] = gridpark_raw['score'] * gridpark_raw['PoP2015_Number']
gridpark_raw['walk_area_ha'] = gridpark_raw['walk_area_m2'] /10000
gridpark_raw['walk_score_ha'] = gridpark_raw['score'] * (gridpark_raw['walk_area_m2'] /10000)
gridpark_raw['walkpop_score'] = gridpark_raw['walk_score_ha'] * gridpark_raw['PoP2015_Number']
gridpark_raw = gridpark_raw.reset_index()

gridpark_raw['Park_No'] = gridpark_raw['Park_No'].fillna(-1)
gridpark_raw['Park_No'] = gridpark_raw['Park_No'].astype(int)

gridpark_raw['Park_entry_No'] = gridpark_raw['Park_entry_No'].fillna(-1)
gridpark_raw['Park_entry_No'] = gridpark_raw['Park_entry_No'].astype(int)

# Gravity model full size
gridpark_grav = routes2.iloc[routes2.groupby('gridpark_no')['grav_total_cost'].idxmin()]
gridpark_grav = gridpark_grav.loc[:, ~gridpark_grav.columns.isin(['raw_total_cost','grav2_total_cost','grav4_total_cost',
                                                'size_infl_proot2','size_infl_proot4'])]
gridpark_grav = gridpark_grav[gridpark_grav['grav_total_cost'] <=1000]
gridpark_grav['score'] = 1000 - gridpark_grav['grav_total_cost']
gridpark_grav = pd.merge(gridpark_grav, popgrid[['PoP2015_Number','geometry']],
                        left_on = 'Grid_No', right_index = True, how = 'outer')
gridpark_grav['score'] = gridpark_grav['score'].fillna(0).round()
gridpark_grav['pop_score'] = gridpark_grav['score'] * gridpark_grav['PoP2015_Number']
gridpark_grav['walk_area_ha'] = gridpark_grav['walk_area_m2'] /10000
gridpark_grav['walk_score_ha'] = gridpark_grav['score'] * (gridpark_grav['walk_area_m2'] /10000)
gridpark_grav['walkpop_score'] = gridpark_grav['walk_score_ha'] * gridpark_grav['PoP2015_Number']
gridpark_grav = gridpark_grav.reset_index()

gridpark_grav['Park_No'] = gridpark_grav['Park_No'].fillna(-1)
gridpark_grav['Park_No'] = gridpark_grav['Park_No'].astype(int)

gridpark_grav['Park_entry_No'] = gridpark_grav['Park_entry_No'].fillna(-1)
gridpark_grav['Park_entry_No'] = gridpark_grav['Park_entry_No'].astype(int)

# Gravity model ** 1/2 size
gridpark_grav2 = routes2.iloc[routes2.groupby('gridpark_no')['grav2_total_cost'].idxmin()]
gridpark_grav2 = gridpark_grav2.loc[:, ~gridpark_grav2.columns.isin(['raw_total_cost','grav_total_cost','grav4_total_cost',
                                                'size_infl_factor','size_infl_proot4'])]
gridpark_grav2 = gridpark_grav2[gridpark_grav2['grav2_total_cost'] <=1000]
gridpark_grav2['score'] = 1000 - gridpark_grav2['grav2_total_cost']
gridpark_grav2 = pd.merge(gridpark_grav2, popgrid[['PoP2015_Number','geometry']],
                        left_on = 'Grid_No', right_index = True, how = 'outer')
gridpark_grav2['score'] = gridpark_grav2['score'].fillna(0).round()
gridpark_grav2['pop_score'] = gridpark_grav2['score'] * gridpark_grav2['PoP2015_Number']
gridpark_grav2['walk_area_ha'] = gridpark_grav2['walk_area_m2'] /10000
gridpark_grav2['walk_score_ha'] = gridpark_grav2['score'] * (gridpark_grav2['walk_area_m2'] /10000)
gridpark_grav2['walkpop_score'] = gridpark_grav2['walk_score_ha'] * gridpark_grav2['PoP2015_Number']
gridpark_grav2 = gridpark_grav2.reset_index()

gridpark_grav2['Park_No'] = gridpark_grav2['Park_No'].fillna(-1)
gridpark_grav2['Park_No'] = gridpark_grav2['Park_No'].astype(int)

gridpark_grav2['Park_entry_No'] = gridpark_grav2['Park_entry_No'].fillna(-1)
gridpark_grav2['Park_entry_No'] = gridpark_grav2['Park_entry_No'].astype(int)

# Gravity model ** 1/4 size
gridpark_grav4 = routes2.iloc[routes2.groupby('gridpark_no')['grav4_total_cost'].idxmin()]
gridpark_grav4 = gridpark_grav4.loc[:, ~gridpark_grav4.columns.isin(['raw_total_cost','grav_total_cost','grav2_total_cost',
                                                'size_infl_factor','size_infl_proot4'])]
gridpark_grav4 = gridpark_grav4[gridpark_grav4['grav4_total_cost'] <=1000]
gridpark_grav4['score'] = 1000 - gridpark_grav4['grav4_total_cost']
gridpark_grav4 = pd.merge(gridpark_grav4, popgrid[['PoP2015_Number','geometry']],
                        left_on = 'Grid_No', right_index = True, how = 'outer')
gridpark_grav4['score'] = gridpark_grav4['score'].fillna(0).round()
gridpark_grav4['pop_score'] = gridpark_grav4['score'] * gridpark_grav4['PoP2015_Number']
gridpark_grav4['walk_area_ha'] = gridpark_grav4['walk_area_m2'] /10000
gridpark_grav4['walk_score_ha'] = gridpark_grav4['score'] * (gridpark_grav4['walk_area_m2'] /10000)
gridpark_grav4['walkpop_score'] = gridpark_grav4['walk_score_ha'] * gridpark_grav4['PoP2015_Number']
gridpark_grav4 = gridpark_grav4.reset_index()

gridpark_grav4['Park_No'] = gridpark_grav4['Park_No'].fillna(-1)
gridpark_grav4['Park_No'] = gridpark_grav4['Park_No'].astype(int)

gridpark_grav4['Park_entry_No'] = gridpark_grav4['Park_entry_No'].fillna(-1)
gridpark_grav4['Park_entry_No'] = gridpark_grav4['Park_entry_No'].astype(int)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')
gridpark_grav4

time taken 0.08 mns


,index,geometry_x,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,...,grav4_total_cost,parkcost_km2,gridpark_no,score,PoP2015_Number,geometry_y,pop_score,walk_area_ha,walk_score_ha,walkpop_score
0,254206.0,"MULTILINESTRING ((-75.00833 40.12855, -75.0087...",normal way,649.034,5.0,10,349,6095,47.215,1.100560e+08,...,753.595473,91.989106,10-349,246.0,127,"POLYGON ((-75.00778 40.13004, -75.00778 40.128...",31242.0,9.626691,2368.165953,3.007571e+05
1,165655.0,"MULTILINESTRING ((-74.99552 40.05713, -74.9964...",normal way,809.681,8.0,1000,35,2262,20.765,1.102507e+08,...,714.259600,109.719346,1000-35,286.0,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",53482.0,24.143101,6904.926994,1.291221e+06
2,165673.0,"LINESTRING (-74.99992 40.05834, -74.99992 40.0...",normal way,0.000,0.0,1000,45,2908,20.765,7.826689e+09,...,17.479737,2.743492,1000-45,983.0,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",183821.0,26.312371,25865.060487,4.836766e+06
3,165694.0,"MULTILINESTRING ((-74.99333 40.05669, -74.9945...",normal way,415.485,4.0,1001,35,2263,58.594,1.104250e+08,...,392.376786,62.635786,1001-35,608.0,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",34656.0,28.155384,17118.473771,9.757530e+05
4,165718.0,"LINESTRING (-74.99565 40.05877, -74.99543 40.0...",normal way,0.000,0.0,1001,45,2910,58.594,1.104250e+08,...,52.845475,7.741497,1001-45,947.0,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",53979.0,19.968856,18910.506705,1.077899e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6399,NaN,None,NaN,NaN,NaN,3886,-1,-1,NaN,NaN,...,NaN,NaN,NaN,0.0,5,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",0.0,NaN,NaN,NaN
6400,NaN,None,NaN,NaN,NaN,3887,-1,-1,NaN,NaN,...,NaN,NaN,NaN,0.0,39,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",0.0,NaN,NaN,NaN
6401,NaN,None,NaN,NaN,NaN,3888,-1,-1,NaN,NaN,...,NaN,NaN,NaN,0.0,5,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",0.0,NaN,NaN,NaN
6402,NaN,None,NaN,NaN,NaN,3889,-1,-1,NaN,NaN,...,NaN,NaN,NaN,0.0,3,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",0.0,NaN,NaN,NaN


In [798]:
gridpark_grav4.columns

Index(['index', 'geometry_x', 'way_calculated', 'route_cost', 'num_steps',
       'Grid_No', 'Park_No', 'Park_entry_No', 'grid_entry_dist',
       'Parkroad_osmid', 'grid_osm', 'walk_area_m2', 'size_infl_proot2',
       'raw euclidean', 'grav4_total_cost', 'parkcost_km2', 'gridpark_no',
       'score', 'PoP2015_Number', 'geometry_y', 'pop_score', 'walk_area_km2',
       'walk_score_km2', 'walkpop_score'],
      dtype='object')

In [ ]:
grid_score = pd.DataFrame(gridpark_raw.groupby('Grid_No')['score','pop_score','walk_area_ha','walkpop_score'].sum())
grid_score.columns = [['raw','raw_pop','Rreach_ha','Rwalkpop']]
grid_score[['grav','grav_pop','Greach_ha','Gwalkpop']] = gridpark_grav.groupby('Grid_No')['score','pop_score','walk_area_ha','walkpop_score'].sum()
grid_score[['grav2','grav2_pop','G2reach_ha','G2walkpop']] = gridpark_grav2.groupby('Grid_No')['score','pop_score','walk_area_ha','walkpop_score'].sum()
grid_score[['grav4','grav4_pop','G4reach_ha','G4walkpop']] = gridpark_grav4.groupby('Grid_No')['score','pop_score','walk_area_ha','walkpop_score'].sum()
grid_score['raw_routes'] = gpd.GeoDataFrame(gridpark_raw[['Grid_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')
grid_score['grav_routes'] = gpd.GeoDataFrame(gridpark_grav[['Grid_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')
grid_score['grav2_routes'] = gpd.GeoDataFrame(gridpark_grav2[['Grid_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')
grid_score['grav4_routes'] = gpd.GeoDataFrame(gridpark_grav4[['Grid_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

grid_score['raw_parks'] = gridpark_raw.groupby('Grid_No')['gridpark_no'].count()
grid_score['grav_parks'] = gridpark_grav.groupby('Grid_No')['gridpark_no'].count()
grid_score['grav2_parks'] = gridpark_grav2.groupby('Grid_No')['gridpark_no'].count()
grid_score['grav4_parks'] = gridpark_grav4.groupby('Grid_No')['gridpark_no'].count()

grid_score['rawPark:entry'] = gridpark_raw[gridpark_raw['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
    ) + ':' + gridpark_raw[gridpark_raw['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
grid_score['gravPark:entry'] = gridpark_grav[gridpark_grav['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
    ) + ':' + gridpark_grav[gridpark_grav['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
grid_score['grav2Park:entry'] = gridpark_grav2[gridpark_grav2['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
    ) + ':' + gridpark_grav2[gridpark_grav2['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
grid_score['grav4Park:entry'] = gridpark_grav2[gridpark_grav2['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
    ) + ':' + gridpark_grav2[gridpark_grav2['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)

grid_score[['raw','raw_pop','Rreach_ha','Rwalkpop','grav','grav_pop','Greach_ha','Gwalkpop','grav2','grav2_pop','G2reach_ha',
            'G2walkpop','grav4','grav4_pop','G4reach_ha','G4walkpop','raw_parks','grav_parks','grav2_parks','grav4_parks',
            'rawPark:entry','gravPark:entry','grav2Park:entry','grav4Park:entry'
           ]].to_csv(r'C:\Users\bartb\Downloads\grid_score.csv')
grid_score


In [850]:
park_score = pd.DataFrame(gridpark_raw.groupby('Park_No')['score','pop_score','walk_area_ha','walkpop_score'].sum())
park_score.columns = [['raw','raw_pop','Rreach_ha','Rwalkpop']]
park_score = park_score[park_score.index >=0]
park_score[['grav','grav_pop','Greach_ha','Gwalkpop']] = gridpark_grav.groupby('Park_No')['score','pop_score','walk_area_ha','walkpop_score'].sum()
park_score[['grav2','grav2_pop','G2reach_ha','G2walkpop']] = gridpark_grav2.groupby('Park_No')['score','pop_score','walk_area_ha','walkpop_score'].sum()
park_score[['grav4','grav4_pop','G4reach_ha','G4walkpop']] = gridpark_grav4.groupby('Park_No')['score','pop_score','walk_area_ha','walkpop_score'].sum()
park_score['raw_routes'] = gpd.GeoDataFrame(gridpark_raw[['Park_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
park_score['grav_routes'] = gpd.GeoDataFrame(gridpark_grav[['Park_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
park_score['grav2_routes'] = gpd.GeoDataFrame(gridpark_grav2[['Park_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
park_score['grav4_routes'] = gpd.GeoDataFrame(gridpark_grav4[['Park_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Park_No')

park_score['raw_grids'] = gridpark_raw.groupby('Park_No')['gridpark_no'].count()
park_score['grav_grids'] = gridpark_grav.groupby('Park_No')['gridpark_no'].count()
park_score['grav2_grids'] = gridpark_grav2.groupby('Park_No')['gridpark_no'].count()
park_score['grav4_grids'] = gridpark_grav4.groupby('Park_No')['gridpark_no'].count()

park_score['rawGrid:P-entry'] = gridpark_raw[gridpark_raw['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
    ) + ':' + gridpark_raw[gridpark_raw['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
park_score['gravGrid:P-entry'] = gridpark_grav[gridpark_grav['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
    ) + ':' + gridpark_grav[gridpark_grav['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
park_score['grav2Grid:P-entry'] = gridpark_grav2[gridpark_grav2['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
    ) + ':' + gridpark_grav2[gridpark_grav2['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
park_score['grav4Grid:P-entry'] = gridpark_grav4[gridpark_grav4['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
    ) + ':' + gridpark_grav2[gridpark_grav4['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)

park_score = park_score.iloc[:,0:len(park_score.columns)].join(parks_within_range['geometry'], how = 'outer')

park_score.columns = ['raw','raw_pop','Rreach_ha','Rwalkpop','grav','grav_pop','Greach_ha','Gwalkpop','grav2','grav2_pop','G2reach_ha',
            'G2walkpop','grav4','grav4_pop','G4reach_ha','G4walkpop','raw_routes','grav_routes','grav2_routes','grav4_routes',
            'raw_grids','grav_grids','grav2_grids','grav4_grids',
            'rawGrid:P-entry', 'gravGrid:P-entry','grav2Grid:P-entry','grav4Grid:P-entry','geometry'
           ]

park_score[['raw','raw_pop','Rreach_ha','Rwalkpop','grav','grav_pop','Greach_ha','Gwalkpop','grav2','grav2_pop','G2reach_ha',
            'G2walkpop','grav4','grav4_pop','G4reach_ha','G4walkpop','raw_grids','grav_grids','grav2_grids','grav4_grids',
            'rawGrid:P-entry', 'gravGrid:P-entry','grav2Grid:P-entry','grav4Grid:P-entry'
           ]].fillna(0).to_csv(r'C:\Users\bartb\Downloads\park_score.csv')

park_score

,raw,raw_pop,Rreach_ha,Rwalkpop,grav,grav_pop,Greach_ha,Gwalkpop,grav2,grav2_pop,...,grav4_routes,raw_grids,grav_grids,grav2_grids,grav4_grids,rawGrid:P-entry,gravGrid:P-entry,grav2Grid:P-entry,grav4Grid:P-entry,geometry
0,18596.0,1739764.0,966.681424,4.251247e+07,51605.0,5814463.0,4834.124156,2.592344e+08,26882.0,2729254.0,...,"(LINESTRING (-75.2410148 40.0578267, -75.24021...",34.0,105.0,57.0,41.0,"[1003, 1004, 1005, 1006, 1053, 1054, 1055, 105...","[1003, 1004, 1005, 1006, 1053, 1054, 1055, 105...","[1003, 1004, 1005, 1006, 1053, 1054, 1055, 105...","[1003, 1004, 1005, 1006, 1053, 1054, 1055, 105...","MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ..."
1,2252.0,625512.0,27.548159,2.633192e+06,6.0,1062.0,5.440536,5.777849e+03,531.0,120351.0,...,"(LINESTRING (-75.2458592 39.91941099999998, -7...",7.0,1.0,2.0,3.0,"[3696, 3697, 3698, 3736, 3737, 3774, 3790]:[27...",[3696]:[27],"[3696, 3697]:[27, 27]","[3696, 3697, 3774]:[27, 27]","POLYGON ((-75.24702 39.91695, -75.24706 39.916..."
2,53090.0,20580640.0,2812.419389,5.155900e+08,126974.0,60041728.0,11035.735245,2.436466e+09,73836.0,31696222.0,...,"(LINESTRING (-75.250656 39.986672000000006, -7...",108.0,259.0,149.0,126.0,"[2720, 2745, 2746, 2747, 2775, 2776, 2805, 280...","[2585, 2615, 2616, 2650, 2651, 2652, 2653, 268...","[2719, 2720, 2745, 2746, 2747, 2748, 2775, 277...","[2719, 2720, 2745, 2746, 2747, 2748, 2775, 277...","MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ..."
3,7768.0,1674466.0,11.547310,8.056491e+05,NaN,NaN,NaN,NaN,468.0,105420.0,...,"(LINESTRING (-75.150406 39.962371, -75.150315 ...",24.0,NaN,2.0,4.0,"[3121, 3122, 3151, 3152, 3176, 3177, 3196, 319...",NaN,"[3196, 3197]:[311, 309]","[3196, 3197, 3228, 3256]:[311, 309]","POLYGON ((-75.14985 39.96195, -75.14993 39.961..."
4,6092.0,1494250.0,21.380191,1.597368e+06,NaN,NaN,NaN,NaN,NaN,NaN,...,(LINESTRING (-75.17361440000002 40.03491760000...,20.0,NaN,NaN,3.0,"[1551, 1552, 1602, 1603, 1658, 1659, 1660, 166...",NaN,NaN,NaN,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.25746 39.94945, -75.25807 39.949..."
470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.27556 39.96885, -75.27571 39.968..."
471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.25665 39.94613, -75.25671 39.946..."
472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.27769 39.96684, -75.27647 39.966..."


In [845]:
park_score.columns

Index([              ('raw',),           ('raw_pop',),         ('Rreach_ha',),
                ('Rwalkpop',),              ('grav',),          ('grav_pop',),
               ('Greach_ha',),          ('Gwalkpop',),             ('grav2',),
               ('grav2_pop',),        ('G2reach_ha',),         ('G2walkpop',),
                   ('grav4',),         ('grav4_pop',),        ('G4reach_ha',),
               ('G4walkpop',),        ('raw_routes',),       ('grav_routes',),
            ('grav2_routes',),      ('grav4_routes',),         ('raw_grids',),
              ('grav_grids',),       ('grav2_grids',),       ('grav4_grids',),
         ('rawGrid:P-entry',),  ('gravGrid:P-entry',), ('grav2Grid:P-entry',),
                   'geometry'],
      dtype='object')

In [578]:
gridpark_lines = gpd.GeoDataFrame(gridpark_score, geometry = 'geometry_x', crs = 4326)
gridpark_nodes = gpd.GeoDataFrame(gridpark_score, geometry = 'geometry_y', crs = 4326)

gridpark_lines[['geometry_x','grav_total_cost','Grid_No','Park_No','PoP2015_Number','pop_score_grav2']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_lines.shp')

gridpark_nodes[['geometry_y','grav_total_cost','Grid_No','Park_No','PoP2015_Number','pop_score_grav2']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_nodes.shp')

In [772]:
grid_prefer = gridpark_grav2.iloc[gridpark_grav2.groupby('Grid_No')['score'].idxmax()]
grid_prefer = grid_prefer[['geometry_x','Parkroad_osmid','grav2_total_cost','Grid_No','Park_No','geometry_y','pop_score']]
grid_prefer = pd.merge(grid_prefer, PtR[['osmid','geometry_x']], left_on = 'Parkroad_osmid', right_on = 'osmid', how = 'left')
lines = list()
for i in range(len(grid_prefer)):
    if grid_prefer['geometry_x_y'].iloc[i] != None:
        lines.append(LineString([grid_prefer['geometry_y'].iloc[i].centroid, grid_prefer['geometry_x_y'].iloc[i]]))
    else: lines.append(None)
grid_prefer['lines'] = lines
#gpd.GeoDataFrame(grid_prefer[['Parkroad_osmid','score','Grid_No','Park_No','pop_score','lines']], 
#                 geometry = 'lines', crs = 4326).to_file(r'C:\Users\bartb\Downloads\gridpark_preference.shp')
grid_prefer

,geometry_x_x,Parkroad_osmid,grav2_total_cost,Grid_No,Park_No,geometry_y,pop_score,osmid,geometry_x_y,lines
0,None,NaN,NaN,0,-1,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",0.0,NaN,None,None
1,None,NaN,NaN,1,-1,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",0.0,NaN,None,None
2,None,NaN,NaN,2,-1,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",0.0,NaN,None,None
3,None,NaN,NaN,3,-1,"POLYGON ((-75.01003 40.13349, -75.01003 40.131...",0.0,NaN,None,None
4,None,NaN,NaN,4,-1,"POLYGON ((-75.00778 40.13349, -75.00778 40.131...",0.0,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...
3886,None,NaN,NaN,3886,-1,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",0.0,NaN,None,None
3887,None,NaN,NaN,3887,-1,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",0.0,NaN,None,None
3888,None,NaN,NaN,3888,-1,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",0.0,NaN,None,None
3889,None,NaN,NaN,3889,-1,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",0.0,NaN,None,None


In [ ]:
park_lines = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'Park access to pop.', 'lines']], 
                              geometry = 'lines', crs = 4326)
park_lines['Park_No'] = park_lines.index
park_lines.to_file(r'C:\Users\bartb\Downloads\park_entrance_routes.shp')

park_s = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'Park access to pop.', 'geometry']], 
                          geometry = 'geometry', crs = 4326)
park_s['Park_No'] = park_lines.index
park_s.to_file(r'C:\Users\bartb\Downloads\park_score_entrance.shp')

In [ ]:
grid_lines = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','lines']], 
                              geometry = 'lines', crs = 4326)
grid_lines.to_file(r'C:\Users\bartb\Downloads\grid_entrance_routes.shp')

grid_s = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','geometry']], 
                          geometry = 'geometry', crs = 4326)
grid_s.to_file(r'C:\Users\bartb\Downloads\grid_score_entrance.shp')

In [185]:
print(round((time.time() - start) / 60,2),'mns')

25.66 mns
